In [279]:
# https://www.bitaddress.org/bitaddress.org-v3.3.0-SHA256-dec17c07685e1870960903d8f58090475b25af946fe95a734f88408cef4aa194.html?testnet=true
#Testnet Address
#Private Key
original_address = 'n2vfdrnUHPLVnf32XTiZHX6e2vvGS1Njxe'
#### private_key = REDACTED

In [ ]:
# https://mempool.space/testnet/address/n2vfdrnUHPLVnf32XTiZHX6e2vvGS1Njxe

In [258]:
from coincurve import PrivateKey as ECPrivateKey, PublicKey as ECPublicKey
# import bitcoin, bit
from hashlib import new, sha256 as _sha256
from io import BytesIO

In [259]:
BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
BASE58_ALPHABET_LIST = list(BASE58_ALPHABET)
BASE58_ALPHABET_INDEX = {char: index for index, char in enumerate(BASE58_ALPHABET)}

In [260]:
# Keys:
MAIN_PRIVATE_KEY = b'\x80'
MAIN_BIP32_PUBKEY = b'\x04\x88\xb2\x1e'
MAIN_BIP32_PRIVKEY = b'\x04\x88\xad\xe4'
TEST_PRIVATE_KEY = b'\xef'
TEST_BIP32_PUBKEY = b'\x045\x87\xcf'
TEST_BIP32_PRIVKEY = b'\x045\x83\x94'
PUBLIC_KEY_UNCOMPRESSED = b'\x04'
PUBLIC_KEY_COMPRESSED_EVEN_Y = b'\x02'
PUBLIC_KEY_COMPRESSED_ODD_Y = b'\x03'
PRIVATE_KEY_COMPRESSED_PUBKEY = b'\x01'

In [261]:
MAIN_PUBKEY_HASH = b'\x00'
MAIN_SCRIPT_HASH = b'\x05'
TEST_PUBKEY_HASH = b'\x6f'
TEST_SCRIPT_HASH = b'\xc4'

In [262]:
def double_sha256(bytestr):
    return _sha256(_sha256(bytestr).digest()).digest()

In [263]:
def double_sha256_checksum(bytestr):
    return double_sha256(bytestr)[:4]

In [264]:
def int_to_unknown_bytes(num, byteorder='big'):
    """Converts an int to the least number of bytes as possible."""
    return num.to_bytes((num.bit_length() + 7) // 8 or 1, byteorder)

In [265]:
def b58decode(string):

    alphabet_index = BASE58_ALPHABET_INDEX

    num = 0

    try:
        for char in string:
            num *= 58
            num += alphabet_index[char]
    except KeyError:
        raise ValueError('"{}" is an invalid base58 encoded character.'.format(char)) from None

    bytestr = int_to_unknown_bytes(num)

    pad = 0
    for char in string:
        if char == '1':
            pad += 1
        else:
            break

    return b'\x00' * pad + bytestr

In [266]:
def b58decode_check(string):

    decoded = b58decode(string)
    shortened = decoded[:-4]
    decoded_checksum = decoded[-4:]
    hash_checksum = double_sha256_checksum(shortened)

    if decoded_checksum != hash_checksum:
        raise ValueError(
            'Decoded checksum {} derived from "{}" is not equal to hash '
            'checksum {}.'.format(decoded_checksum, string, hash_checksum)
        )

    return shortened

In [267]:
def wif_to_bytes(wif):

    private_key = b58decode_check(wif)

    version = private_key[:1]

    if version == MAIN_PRIVATE_KEY:
        version = 'main'
    elif version == TEST_PRIVATE_KEY:
        version = 'test'
    else:
        raise ValueError('{} does not correspond to a mainnet nor testnet address.'.format(version))

    # Remove version byte and, if present, compression flag.
    if len(wif) == 52 and private_key[-1] == 1:
        private_key, compressed = private_key[1:-1], True
    else:
        private_key, compressed = private_key[1:], False

    return private_key, compressed, version

In [268]:
from collections import namedtuple

FIELD_SIZE = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
GROUP_ORDER = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
TONELLI_SHANKS_CONSTANT = (FIELD_SIZE + 1) // 4


Point = namedtuple('Point', ('x', 'y'))


def parity(num):
    return num & 1


def x_to_y(x, y_parity):

    y = pow(x ** 3 + 7, TONELLI_SHANKS_CONSTANT, FIELD_SIZE)

    if parity(y) != y_parity:
        y = FIELD_SIZE - y

    return y

In [269]:
def public_key_to_coords(public_key):

    length = len(public_key)

    if length == 33:
        flag, x = int.from_bytes(public_key[:1], 'big'), int.from_bytes(public_key[1:], 'big')
        y = x_to_y(x, flag & 1)
    elif length == 65:
        x, y = int.from_bytes(public_key[1:33], 'big'), int.from_bytes(public_key[33:], 'big')
    else:
        raise ValueError('{} is an invalid length for a public key.'.format(length))

    return x, y


In [270]:
def public_point(_public_key,_public_point=None):
    """The public point (x, y)."""
    if _public_point is None:
        _public_point = Point(*public_key_to_coords(_public_key))
    return _public_point

In [271]:
private_key_bytes = wif_to_bytes(private_key)
_pk = PrivateKey(private_key_bytes[0])

In [272]:
_public_key = _pk.public_key.format(compressed=private_key_bytes[1])

In [273]:
_public_key

b'\x03\xbb\xbfp\xa9\x85F\x8e\xc9\x8fE\x9a\x07\x92\xcfv\x84\x9aU\xe6\xec\xf2\xce\xfdj=\xfc\x08\xffQ\xcfp\xa3'

In [274]:
public_point(_public_key)

Point(x=84920748040300873896448322609322126171245396536779421472866519299972580470947, y=115327645188757170880311840198543563942798424331185797480365562124129996929519)

In [275]:
def ripemd160_sha256(bytestr):
    return new('ripemd160', sha256(bytestr).digest()).digest()

In [250]:
from collections import deque

In [251]:
def b58encode(bytestr):

    alphabet = BASE58_ALPHABET_LIST

    encoded = deque()
    append = encoded.appendleft
    _divmod = divmod

    num = int.from_bytes(bytestr, 'big')

    while num > 0:
        num, rem = _divmod(num, 58)
        append(alphabet[rem])

    encoded = ''.join(encoded)

    pad = 0
    for byte in bytestr:
        if byte == 0:
            pad += 1
        else:
            break

    return '1' * pad + encoded


In [276]:
def b58encode_check(bytestr):
    return b58encode(bytestr + double_sha256_checksum(bytestr))

In [277]:
def public_key_to_address(public_key, version='main'):

    if version == 'test':
        version = TEST_PUBKEY_HASH
    else:
        version = MAIN_PUBKEY_HASH

    length = len(public_key)

    if length not in (33, 65):
        raise ValueError('{} is an invalid length for a public key.'.format(length))

    return b58encode_check(version + ripemd160_sha256(public_key))

In [278]:
b58encode_check(TEST_PUBKEY_HASH + ripemd160_sha256(_public_key))

'n2vfdrnUHPLVnf32XTiZHX6e2vvGS1Njxe'